In [1]:
from tqdm import tqdm
tqdm.pandas()
from glob import glob

import json
import csv
import numpy as np
import pandas as pd
from collections import Counter

import rdflib
from rdflib import Graph
from data.data import CollectionAccessor, ImageHandler, get_latest

from search import Search, Randomiser

/home/valentin/home2-env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:

# dmg_meta = dict(name="DMG 2025-04-03", id_="DMG_2025-04-03",
#                 creation_timestamp="2025-04-03")
# # dmg = CollectionAccessor.get_DMG("./data/DMG_2025-03-26.csv", "", **dmg_meta)
# image_folder = "./data/images/DMG"
# img = ImageHandler(image_folder=image_folder)

# dmg = CollectionAccessor.get_DMG(pub_path="./data/dumps/API_dump_public_2025-04-03_extracted.csv",
#                                  priv_path="./data/dumps/API_dump_private_2025-04-03_extracted.csv",
#                                  rights_path="./data/rights.csv",
#                                  image_handler=img,
#                                  **dmg_meta)

In [5]:
def init_DMG():
    image_folder = "./data/images/DMG"
    image_handler = ImageHandler(image_folder=image_folder, keep_prefix=False)

    dmg_meta = dict(name="Design Museum Gent (public & private)", id_="DMG_2025-05-06",
                creation_timestamp="2025-05-06")
    df = CollectionAccessor.get_DMG(pub_path=get_latest("./data/dumps", contains="public"),
                                     priv_path=get_latest("./data/dumps", contains="private"),
                                     rights_path="./data/rights.csv",
                                     image_handler=image_handler,
                                     **dmg_meta)
    
    rand = Randomiser(df, name="R1")
    rand2 = Randomiser(df, name="R2")
    s = Search([rand, rand2])
    return image_handler, df, s

im, dmg, dmg_search = init_DMG()

100%|█████████████████████████████████████████████████████████████████████| 36458/36458 [00:00<00:00, 486635.42it/s]


In [6]:
dmg.coll.get_presentation_records(object_numbers=list(dmg.sample(4).index))
# for i, r in sub.iterrows():
#     r.image_path

[{'inventory_number': '2017-0314_2-5',
  'title': "Hulpstuk van het massageapparaat '77I'",
  'description': 'Het lichtblauwe massageapparaat van Nova werd geleverd met drie hulpstukken, waaronder deze trilplaat met noppen voor de behandeling van cellulitis. In de jaren 1960 kregen gezondheid, welzijn en uiterlijke schoonheid meer en meer aandacht. Een slanke taille, lange benen en een weelderige boezem wer …',
  'designer': '',
  'producer': 'Nova',
  'design_date': '',
  'production_date': 'na ca. 1972',
  'design_place': '',
  'production_place': 'Tongeren',
  'rights_attribution': 'In Copyright',
  'image_path': '/2017-0314_0-5/2017-0314_2-5$1.jpg'},
 {'inventory_number': '2017-0314_2-5',
  'title': "Hulpstuk van het massageapparaat '77I'",
  'description': 'Het lichtblauwe massageapparaat van Nova werd geleverd met drie hulpstukken, waaronder deze trilplaat met noppen voor de behandeling van cellulitis. In de jaren 1960 kregen gezondheid, welzijn en uiterlijke schoonheid meer en m

In [ ]:
dmg.image_path.isna().sum()

In [ ]:
dmg["objectname_label"]

---
# DEV FUNCTIONS

In [ ]:
def parse_id_list(id_list_str):
    try:
        return list(map(str.strip, id_list_str.split(",")))
    except ValueError:
        raise s

In [ ]:
def search_collection(collection_id, object_ids, concept, model_list):
    cur_coll = get_collection(collection_id)
    object_ids = parse_id_list(object_ids)
    model_list = parse_id_list(model_list)
    cur_search = searches[collection_id]
    print(object_ids)
    scores = cur_search(object_ids)


object_ids = "1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"
object_ids = parse_id_list(object_ids)
object_ids
recs = dmg.loc[object_ids]

In [ ]:
# searcher_scores = [cur_s(recs) for cur_s in dmg_search.searchers]
# searcher_scores = pd.DataFrame({cur_s.name: cur_s for cur_s in searcher_scores})
# searcher_scores.loc[recs.index] = 0.


# searcher_scores.round(3)

---
# TESTING ROUTES

In [ ]:
!curl "http://0.0.0.0:8080/collections"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/object-details?object_ids=1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/models"

In [ ]:
!curl "http://0.0.0.0:8080/moon?ISO_8601_datetime=2024-05-13&lat_degrees=51.05&long_degrees=3.71"

In [ ]:
# default params for search
!curl "http://0.0.0.0:8080/DMG_2025-05-06/search"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/search?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b""

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/search/order?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b"&skip=200&limit=100&reverse=true"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/search/sample?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b"&k=100"

In [ ]:
!curl "http://0.0.0.0:8080/DMG_2025-05-06/search/order/filter?object_ids="1987-0120_04-14,1987-1343_2-4,3370,3703_0-2"&concept="hello"&model_list="a,b"&filter_text="doos""